In [16]:
import json
import pandas as pd
import re

from glob import glob

In [35]:
path = "data/Italy/2022\\010101101110812620.txt"



In [59]:
output = []
output_location = []
for path in glob("data/Italy/2022/*.txt"):
    
    try:

        file = open(path, "r", encoding="utf-8")
        data = json.load(file)
        details = data["int"]
        cand = data["cand"]

        file.close()

        polling_id = re.findall(r"[^\/]+(?=\.)", path)[0][5:]

        tmp = pd.DataFrame(cand).explode("liste")

        df = pd.json_normalize(tmp["liste"])
        df["polling_id"] = polling_id
        df = df.rename(columns={
            "desc_lis": "candidate",
            "voti": "value"
        })

        output.append(df)
        output_location.append({
            "commune_id": details["cod_com"],
            "commune": details["desc_com"],
            "province_id": details["cod_prov"],
            "province": details["desc_prov"],
            "polling_id": polling_id,
            "circunscription_id": details["cod_circ"],
            "circunscription": details["desc_circ"],
            "dt_agg": details["dt_agg"]
        })
        
    except:
#         pass
        print(path)
    
df = pd.concat(output, ignore_index=True)
df.head()

data/Italy/2022\022700027010040010.txt
data/Italy/2022\022700027010040020.txt
data/Italy/2022\022700027010040030.txt
data/Italy/2022\022700027010040040.txt
data/Italy/2022\022700027010040050.txt
data/Italy/2022\022700027010040060.txt
data/Italy/2022\022700027010040070.txt
data/Italy/2022\022700027010040080.txt
data/Italy/2022\022700027010040090.txt
data/Italy/2022\022700027010040100.txt
data/Italy/2022\022700027010040110.txt
data/Italy/2022\022700027010040120.txt
data/Italy/2022\022700027010040130.txt
data/Italy/2022\022700027010040140.txt
data/Italy/2022\022700027010040150.txt
data/Italy/2022\022700027010040160.txt
data/Italy/2022\022700027010040170.txt
data/Italy/2022\022700027010040180.txt
data/Italy/2022\022700027010040190.txt
data/Italy/2022\022700027010040200.txt
data/Italy/2022\022700027010040210.txt
data/Italy/2022\022700027010040220.txt
data/Italy/2022\022700027010040230.txt
data/Italy/2022\022700027010040240.txt
data/Italy/2022\022700027010040250.txt
data/Italy/2022\022700027

,pos,candidate,img_lis,value,perc,polling_id
0,13,PARTITO DEMOCRATICO - ITALIA DEMOCRATICA E PRO...,2-20220925000000-13.png,51930,"26,22",010101101110812620
1,17,ALLEANZA VERDI E SINISTRA,2-20220925000000-17.png,14139,"7,14",010101101110812620
2,16,+EUROPA,2-20220925000000-16.png,12048,"6,08",010101101110812620
3,9,IMPEGNO CIVICO LUIGI DI MAIO - CENTRO DEMOCRATICO,2-20220925000000-9.png,813,"0,41",010101101110812620
4,14,FRATELLI D'ITALIA CON GIORGIA MELONI,2-20220925000000-14.png,39003,"19,69",010101101110812620


In [53]:
df_filtered = df[["value", "polling_id", "candidate"]].copy()
df_filtered["rank"] = df_filtered.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)

tt = df_filtered.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate"]]

df_filtered = pd.merge(df_filtered, tt, on=["polling_id", "candidate"])
df_filtered.head()

,value,polling_id,candidate,rank,rate
0,51930,010101101110812620,PARTITO DEMOCRATICO - ITALIA DEMOCRATICA E PRO...,1,0.262187
1,14139,010101101110812620,ALLEANZA VERDI E SINISTRA,5,0.071386
2,12048,010101101110812620,+EUROPA,6,0.060829
3,813,010101101110812620,IMPEGNO CIVICO LUIGI DI MAIO - CENTRO DEMOCRATICO,14,0.004105
4,39003,010101101110812620,FRATELLI D'ITALIA CON GIORGIA MELONI,2,0.196920


In [54]:
df_filtered.to_csv(f"data_output/Italy/2022_first_round.csv.gz", compression="gzip", index=False)

In [57]:
df_location = pd.DataFrame(output_location)

df_location.to_csv(f"data_output/Italy/2022_first_round_location.csv.gz", compression="gzip", index=False)